# Example: Metabolite workflow

This notebook provides insights on how to use the tool if you have list of metabolites.
Our current data sources include:
* MolMeDB

In [1]:
# Import modules
import pandas as pd

from pyBiodatafuse import id_mapper
from pyBiodatafuse.annotators import opentargets, molmedb
from pyBiodatafuse.utils import combine_sources

# Load list of metabolites/chemicals/compounds

In [2]:
metabolites_of_interest = """100208
10040286
10041551
10025195
5291"""
# metabolites_of_interest = """CHEMBL1201583
# CHEMBL941
# """
# metabolites_of_interest = """5291"""
metabolite_list = metabolites_of_interest.split("\n")
len(metabolite_list)

5

In [3]:
data_input = pd.DataFrame(metabolite_list, columns=["identifier"])
data_input.head()

,identifier
0,100208
1,10040286
2,10041551
3,10025195
4,5291


In [4]:
bridgdb_df, bridgdb_metadata = id_mapper.bridgedb_xref(
    identifiers=data_input,
    input_species="Human",
    input_datasource="PubChem Compound", #ChEMBL compound
    output_datasource="All",
)
bridgdb_df.head()

,identifier,identifier.source,target,target.source
0,100208,PubChem-compound,90560,ChemSpider
1,100208,PubChem-compound,100208,PubChem Compound
2,100208,PubChem-compound,HMDB0244377,HMDB
3,100208,PubChem-compound,OFDNQWIFNXBECV-UHFFFAOYSA-N,InChIKey
4,100208,PubChem-compound,C11280,KEGG Compound


### Diseases from OpenTarget

In [5]:
disease_df, opentargets_disease_metadata = opentargets.get_compound_disease_interactions(
    bridgedb_df=bridgdb_df
)
disease_df.head()

,identifier,identifier.source,target,target.source,OpenTargets_Diseases
0,100208,PubChem-compound,100208,PubChem Compound,"[{'disease_id': nan, 'disease_name': nan, 'the..."
1,10025195,PubChem-compound,10025195,PubChem Compound,"[{'disease_id': nan, 'disease_name': nan, 'the..."
2,10040286,PubChem-compound,10040286,PubChem Compound,"[{'disease_id': nan, 'disease_name': nan, 'the..."
3,10041551,PubChem-compound,10041551,PubChem Compound,"[{'disease_id': nan, 'disease_name': nan, 'the..."
4,5291,PubChem-compound,5291,PubChem Compound,"[{'disease_id': 'umls:C0009375', 'disease_name..."


In [6]:
opentargets_disease_metadata

{'datasource': 'Open Targets GraphQL & REST API Beta',
 'metadata': {'source_version': {'apiVersion': {'x': '24',
    'y': '1',
    'z': '4'}},
  'data_version': {'dataVersion': {'year': '24', 'month': '06'}}},
 'query': {'size': 3,
  'input_type': 'PubChem Compound',
  'number_of_added_nodes': 105,
  'number_of_added_edges': 127,
  'time': '0:00:00.066203',
  'date': '2024-08-07 14:13:47',
  'url': 'https://api.platform.opentargets.org/api/v4/graphql'}}

In [7]:
disease_df["OpenTargets_Diseases"][4]

[{'disease_id': 'umls:C0009375',
  'disease_name': 'colonic neoplasm',
  'therapeutic_areas': 'MONDO_0045024:cancer or benign tumor, EFO_0010282:gastrointestinal disease'},
 {'disease_id': 'umls:C0238033, umls:C0242787, umls:C0242788',
  'disease_name': 'male breast carcinoma',
  'therapeutic_areas': 'OTAR_0000017:reproductive system or breast disease, MONDO_0045024:cancer or benign tumor, EFO_0010285:integumentary system disease'},
 {'disease_id': 'umls:C0007115',
  'disease_name': 'thyroid cancer',
  'therapeutic_areas': 'EFO_0001379:endocrine system disease, MONDO_0045024:cancer or benign tumor'},
 {'disease_id': 'umls:C0036421',
  'disease_name': 'systemic scleroderma',
  'therapeutic_areas': 'OTAR_0000006:musculoskeletal or connective tissue disease, OTAR_0000018:genetic, familial or congenital disease, OTAR_0000010:respiratory or thoracic disease, EFO_0009690:urinary system disease, MONDO_0024458:disorder of visual system, EFO_0000540:immune system disease, EFO_0010285:integument

In [8]:
transporter_inhibited_df, transporter_inhibited_metadata = molmedb.get_compound_gene_inhibitor(
    bridgedb_df=bridgdb_df
)
transporter_inhibited_df.head()

,identifier,identifier.source,target,target.source,MolMeDB_transporter_inhibited
0,100208,PubChem-compound,OFDNQWIFNXBECV-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': 'P08183', 'hgnc_symbol'..."
1,10025195,PubChem-compound,LEJRLSZVESQKJK-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': 'Q01959', 'hgnc_symbol'..."
2,10040286,PubChem-compound,FYGREZKTJIXWIH-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': 'Q01959', 'hgnc_symbol'..."
3,10041551,PubChem-compound,OVVBIIBBRZVPAL-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': 'P23975', 'hgnc_symbol'..."
4,5291,PubChem-compound,KTUFNOKKBVMGRW-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': 'O15244', 'hgnc_symbol'..."


In [9]:
transporter_inhibited_metadata

{'datasource': 'MolMeDB',
 'query': {'size': 149,
  'input_type': 'InChIKey',
  'number_of_added_nodes': 9,
  'number_of_added_edges': 12,
  'time': '0:00:00.388044',
  'date': '2024-08-07 14:13:57',
  'url': 'https://idsm.elixir-czech.cz/sparql/endpoint/molmedb'}}

In [10]:
transporter_inhibited_df["MolMeDB_transporter_inhibited"][4]

[{'uniprot_trembl_id': 'O15244',
  'hgnc_symbol': 'SLC22A2',
  'source_doi': 'doi:10.1021/jm301302s',
  'source_pmid': '23241029'},
 {'uniprot_trembl_id': 'O15245',
  'hgnc_symbol': 'SLC22A1',
  'source_doi': 'doi:10.1021/jm301302s',
  'source_pmid': '23241029'},
 {'uniprot_trembl_id': 'O75751',
  'hgnc_symbol': 'SLC22A3',
  'source_doi': 'doi:10.1021/jm301302s',
  'source_pmid': '23241029'},
 {'uniprot_trembl_id': 'Q92887',
  'hgnc_symbol': 'ABCC2',
  'source_doi': 'doi:10.1093/toxsci/kft176',
  'source_pmid': '23956101'},
 {'uniprot_trembl_id': 'Q9UNQ0',
  'hgnc_symbol': 'ABCG2',
  'source_doi': 'doi:10.1124/mol.65.6.1485',
  'source_pmid': '15155841'},
 {'uniprot_trembl_id': 'Q9UNQ0',
  'hgnc_symbol': 'ABCG2',
  'source_doi': 'doi:10.1016/j.bmc.2008.07.034',
  'source_pmid': '18678495'},
 {'uniprot_trembl_id': 'Q9UNQ0',
  'hgnc_symbol': 'ABCG2',
  'source_doi': 'doi:10.1016/j.bmcl.2009.11.004',
  'source_pmid': '19932960'}]